In [2]:
import yfinance as yf

msft = yf.Ticker("MSFT")

# get all stock info
msft.info

# get historical market data
hist = msft.history(period="1mo")
print(hist)

f:\MarketInsightsGenerator\venv\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


                                 Open        High         Low       Close  \
Date                                                                        
2024-01-22 00:00:00-05:00  399.281607  399.880506  392.863483  395.778107   
2024-01-23 00:00:00-05:00  395.019517  398.642821  393.202869  398.163696   
2024-01-24 00:00:00-05:00  400.798819  404.881266  399.710834  401.816925   
2024-01-25 00:00:00-05:00  403.573708  406.258745  401.787004  404.122681   
2024-01-26 00:00:00-05:00  403.623608  405.420304  401.687186  403.184418   
2024-01-29 00:00:00-05:00  405.310467  409.223244  403.583649  408.963715   
2024-01-30 00:00:00-05:00  411.499055  412.287575  405.699781  407.835815   
2024-01-31 00:00:00-05:00  406.208822  414.553406  396.476818  396.846130   
2024-02-01 00:00:00-05:00  401.088286  407.246910  401.058342  403.034698   
2024-02-02 00:00:00-05:00  403.064642  411.888321  402.815103  410.460968   
2024-02-05 00:00:00-05:00  409.143400  410.401085  403.244305  404.901245   

In [3]:
import yfinance as yf


def get_stock_data(symbol):
    # Create a Ticker object for the given stock symbol
    stock = yf.Ticker(symbol)

    # Get historical market data for the last one month
    hist_data = stock.history(period="1mo")

    return hist_data


# Example usage
stock_symbol = "MSFT"
stock_data = get_stock_data(stock_symbol)
print(stock_data)

                                 Open        High         Low       Close  \
Date                                                                        
2024-01-22 00:00:00-05:00  399.281607  399.880506  392.863483  395.778107   
2024-01-23 00:00:00-05:00  395.019517  398.642821  393.202869  398.163696   
2024-01-24 00:00:00-05:00  400.798819  404.881266  399.710834  401.816925   
2024-01-25 00:00:00-05:00  403.573708  406.258745  401.787004  404.122681   
2024-01-26 00:00:00-05:00  403.623608  405.420304  401.687186  403.184418   
2024-01-29 00:00:00-05:00  405.310467  409.223244  403.583649  408.963715   
2024-01-30 00:00:00-05:00  411.499055  412.287575  405.699781  407.835815   
2024-01-31 00:00:00-05:00  406.208822  414.553406  396.476818  396.846130   
2024-02-01 00:00:00-05:00  401.088286  407.246910  401.058342  403.034698   
2024-02-02 00:00:00-05:00  403.064642  411.888321  402.815103  410.460968   
2024-02-05 00:00:00-05:00  409.143400  410.401085  403.244305  404.901245   

f:\MarketInsightsGenerator\venv\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')


In [4]:
import yfinance as yf
import psycopg2

hostname = "localhost"
database = "Finance"
username = "postgres"
pwd = "1234"
port_id = 5432

conn = None
curr = None

try:
    # Establish connection to PostgreSQL
    conn = psycopg2.connect(
        host=hostname, dbname=database, user=username, password=pwd, port=port_id
    )
    curr = conn.cursor()

    # Create the stock table if it doesn't exist
    create_script = """CREATE TABLE IF NOT EXISTS stock (
                    date_time TIMESTAMP PRIMARY KEY,
                    open_price FLOAT,
                    high_price FLOAT,
                    low_price FLOAT,
                    close_price FLOAT
                    )"""
    curr.execute(create_script)
    conn.commit()

    # Get historical market data for the last one month
    msft = yf.Ticker("MSFT")
    hist = msft.history(period="1mo")

    # Insert data into the stock table
    for index, row in hist.iterrows():
        curr.execute(
            "INSERT INTO stock (date_time, open_price, high_price, low_price, close_price) VALUES (%s, %s, %s, %s, %s)",
            (index, row["Open"], row["High"], row["Low"], row["Close"]),
        )
    conn.commit()

except Exception as error:
    print(error)

finally:
    if curr is not None:
        curr.close()
    if conn is not None:
        conn.close()

f:\MarketInsightsGenerator\venv\lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
